In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tf-keras --quiet
!pip install gensim --quiet
!pip install tensorflow-text --quiet
!pip install altair_data_server --quiet
!pip install -q evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
ERROR

In [ ]:
#import libraries
import pandas as pd
import os
from google.colab import drive

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras

from keras.layers import Embedding, Input, Dense, Lambda
from keras.models import Model
#import keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import sklearn as sk
from sklearn.model_selection import train_test_split
import os
import nltk
from nltk.data import find
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import transformers
import torch
import evaluate
from functools import partial

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

import altair as alt
from altair_data_server import data_server

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install datasets --quiet

In [ ]:
from transformers import T5Tokenizer, TFT5Model, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, load_dataset

In [ ]:
chunk_highlights = pd.read_csv('/content/drive/MyDrive/266 Final Project/Processed Data/chunk_highlights.csv')
chunk_highlights = chunk_highlights.dropna()

In [ ]:
#cleaning /n - breaking something - creating empty cells?
# chunk_highlights['doc_text'] = chunk_highlights['doc_text'].apply(lambda x: x.strip())
# chunk_highlights['highlight'] = chunk_highlights['highlight'].apply(lambda x: x.strip())
# chunk_highlights['doc_text'] = chunk_highlights['doc_text'].apply(lambda x: x.replace(r'\n', ' '))
# chunk_highlights['highlight'] = chunk_highlights['highlight'].apply(lambda x: x.replace(r'\n', ' ')) #try \\n instead of r'\n'

In [ ]:
# !!!!! CREATING SMALLER DF FOR TESTING !!!!!!
# balanced_df_test = balanced_df[0:1000]

# Create Train and Test data split
X = chunk_highlights['doc_text'].values
y = chunk_highlights['highlight'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print(X_train.shape)
print(X_test.shape)

train_data_df = pd.DataFrame(columns = ('X','y'))
train_data_df['X'] = X_train
train_data_df['y'] = y_train
test_data_df = pd.DataFrame(columns = ('X','y'))
test_data_df['X'] = X_test
test_data_df['y'] = y_test

train_filepath = '/content/drive/MyDrive/266 Final Project/Processed Data/train_data_chunks_NL.csv'
test_filepath = '/content/drive/MyDrive/266 Final Project/Processed Data/test_data_chunks_NL.csv'

#write training and test data
pd.DataFrame(train_data_df).to_csv(train_filepath, index = False)
pd.DataFrame(test_data_df).to_csv(test_filepath, index = False)

#save for trainer and clear memory
num_train_examples = len(train_data_df)

#probably need to clear other variables too for this to be effective
train_data_df = None
# test_data_df = None

# Create the datasets for train and validation data
train_dataset = load_dataset("csv", data_files=train_filepath, streaming=True)['train']
test_dataset = load_dataset("csv", data_files=test_filepath, streaming=True)['train']

(4739,)
(1185,)


In [ ]:
#for sequence classification
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: t5-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
#for baseline
t5_base = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
train_dataset = load_dataset("csv", data_files=train_filepath, streaming=True)['train']
test_dataset = load_dataset("csv", data_files=test_filepath, streaming=True)['train']

for example in train_dataset:
  print(example)
  print(example['X'])
  print(example['y'])
  break

def t5_summarization_tokenizer(df, MAX_LENGTH=256):
  prompt = 'summarize: '
  inputs = df['X']
  targets = df['y']

  tokenized_inputs = t5_tokenizer.batch_encode_plus([prompt + inputs], #never included prompt but it seems to be working????
                                  padding='max_length',
                                  truncation=True,
                                  max_length=MAX_LENGTH,
                                  return_attention_mask = True,
                                  return_tensors="pt"
  )


  # Set the target (summary) as well
  tokenized_labels = t5_tokenizer.batch_encode_plus([targets],
                        padding='max_length',
                        max_length=MAX_LENGTH,
                        truncation=True,
                                                    return_tensors = 'pt')

  # Create a Dataset from the tokenized inputs and labels
  tokenized_text = {
      "input_ids": tokenized_inputs["input_ids"][0],
      "attention_mask": tokenized_inputs["attention_mask"][0],
      "labels": tokenized_labels["input_ids"][0]
  }

  return tokenized_text

train_dataset = train_dataset.map(t5_summarization_tokenizer)
test_dataset = test_dataset.map(t5_summarization_tokenizer)


{'X': 'in the App or on our web site, or through a separate agreement with Kahoot!. If you have signed up for a Paid Subscription for a monthly subscription fee, you will be charged for one months subscription fee in advance upon expiry of any Trial. The Paid Subscription and the payment to Kahoot!', 'y': 'The Paid Subscription and the payment to Kahoot!'}
in the App or on our web site, or through a separate agreement with Kahoot!. If you have signed up for a Paid Subscription for a monthly subscription fee, you will be charged for one months subscription fee in advance upon expiry of any Trial. The Paid Subscription and the payment to Kahoot!
The Paid Subscription and the payment to Kahoot!


In [ ]:
for example in train_dataset:
  print(example)
  break

{'X': 'in the App or on our web site, or through a separate agreement with Kahoot!. If you have signed up for a Paid Subscription for a monthly subscription fee, you will be charged for one months subscription fee in advance upon expiry of any Trial. The Paid Subscription and the payment to Kahoot!', 'y': 'The Paid Subscription and the payment to Kahoot!', 'input_ids': tensor([21603,    10,    16,     8,  2276,    42,    30,    69,   765,   353,
            6,    42,   190,     3,     9,  2450,  2791,    28,  2209,   107,
           32,    32,    17,    55,     5,   156,    25,    43,  3814,    95,
           21,     3,     9, 14697,    26,  3325, 11830,    21,     3,     9,
         3718,  7644,  2572,     6,    25,    56,    36,  4977,    21,    80,
          767,  7644,  2572,    16,  3245,  1286,  8982,    63,    13,   136,
        20660,     5,    37, 14697,    26,  3325, 11830,    11,     8,  1942,
           12,  2209,   107,    32,    32,    17,    55,     1,     0,     0,
    

In [ ]:
batch_size = 5
num_epochs = 3

#save model for fine - tune
dir_path = '/content/drive/MyDrive/266 Final Project/model_checkpoints/'
file_path = dir_path + 't5base-finetuned-summarization-chunks'

training_args = TrainingArguments(
    output_dir='./results',
    #output_dir= file_path,          # output directory
    evaluation_strategy="epoch",     # evaluation strategy to adopt during training
    learning_rate=1e-5,              # learning rate
    per_device_train_batch_size=batch_size,   # batch size for training
    per_device_eval_batch_size=batch_size,    # batch size for evaluation
    max_steps = int(num_epochs * num_train_examples / batch_size),
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    report_to="none"        # Important, otherwise will try to connect and write to wandb
)

trainer = Trainer(model=t5_model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss
0,1.116000,0.018606
1,0.029100,0.015566
2,0.025200,0.015006


TrainOutput(global_step=2843, training_loss=0.21941882944258007, metrics={'train_runtime': 2250.638, 'train_samples_per_second': 6.316, 'train_steps_per_second': 1.263, 'total_flos': 4327559249264640.0, 'train_loss': 0.21941882944258007, 'epoch': 2.333098839254309})

In [ ]:
def generate_t5_summary(text): #maybe batch .map this function in the future?
    # Tokenize the input text
    prompt = 'summarize: '
    inputs = t5_tokenizer(prompt + text, return_tensors="pt", max_length=256, truncation=True)

    # Move inputs to the same device as the model
    device = t5_model.device  # Get the device of the model #use t5_base for baseline
    inputs = {k: v.to(device) for k, v in inputs.items()} # Move all tensors in inputs to the device

    # Generate summary
    outputs = t5_model.generate(**inputs, max_new_tokens = 100) #less values -> worse performance (chopped off sentences) - still needs a value to generate words

    # Decode the generated summary
    summary = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Test T5 summary generation
test_data_df['t5_extract'] = test_data_df['X'].apply(lambda x: generate_t5_summary(x))

In [ ]:
#baseline - wanted to run this without training model first but its taking a long time for some reason???
# test_data_df['t5_extract'] = test_data_df['X'].apply(lambda x: generate_t5_summary(x))
# baseline_filepath = '/content/drive/MyDrive/266 Final Project/Processed Data/baseline_chunks_NL.csv'
# pd.DataFrame(test_data_df).to_csv(baseline_filepath, index = False)

KeyboardInterrupt: 

In [ ]:
def generate_t5_summary_batch(text):
  "Testing for batch processing for efficiency WIP"
  inputs = text['X']
  prompt = 'summarize: '
  inputs = t5_tokenizer.batch_encode_plus([prompt + inputs],
                                  truncation=True,
                                  max_length=256,
                                  return_tensors="pt")
  device = t5_model.device
  inputs = {k: v.to(device) for k, v in inputs.items()} # Move all tensors in inputs to the device

  outputs = t5_model.generate(**inputs, max_new_tokens = 100)
  summary = t5_tokenizer.decode(outputs[0],skip_special_tokens = True)

  # Create a Dataset from the tokenized inputs and labels
  results = {
      "summary": summary
  }

  return results

In [ ]:
#apply generate summary to batch and save
test_dataset = test_dataset.map(generate_t5_summary_batch)

In [ ]:
for example in test_dataset:
  print(example)
  break

{'X': 'Please ask if you have any questions. IndemnityTo summarize If we are sued because of your use of or conduct on Patreon, you have to help pay for it. You will indemnify us from all losses and liabilities, including legal fees, that arise from these terms or relate to your use of Patreon.', 'y': 'You will indemnify us from all losses and liabilities, including legal fees, that arise from these terms or relate to your use of Patreon.', 'input_ids': tensor([21603,    10,   863,   987,     3,    99,    25,    43,   136,   746,
            5,    86,  1778,    29,   485,  3696, 21603,   156,    62,    33,
        19557,   250,    13,    39,   169,    13,    42,  3498,    30,  5192,
           60,   106,     6,    25,    43,    12,   199,   726,    21,    34,
            5,   148,    56, 11468,    29,  4921,   178,    45,    66,  8467,
           11, 25468,     6,   379,  1281,  3051,     6,    24,  7931,    45,
          175,  1353,    42,  9098,    12,    39,   169,    13,  5192,    

In [ ]:
import csv
output_file = '/content/drive/MyDrive/266 Final Project/Processed Data/chunk_batch_testing_NL.csv'
with open(output_file, mode = 'w', newline = '', encoding = 'utf-8') as f:
  writer = csv.DictWriter(f, fieldnames = ['X', 'y','summary'])
  writer.writeheader()

for batch in test_dataset:
  result = generate_t5_summary_batch(batch)
  with open(output_file, mode = 'a', newline = '',encoding = 'utf-8') as f:
    writer = csv.DictWriter(f, fieldnames = ['X','y','summary'])
    writer.writerow(result)

KeyboardInterrupt: 

In [ ]:
results_filepath = '/content/drive/MyDrive/266 Final Project/Processed Data/chunk_testing_NL.csv'
pd.DataFrame(test_data_df).to_csv(results_filepath, index = False)

In [ ]:
#t5_seqclass_model_saved = T5ForSequenceClassification.from_pretrained(file_path + '/checkpoint-2500') #2500 11/15/2024

#Evaluation

In [ ]:
#evaluate with ROUGE based on results_filepath file
#need to try a baseline (without training)?

In [ ]:
# baseline_df = pd.read_csv(baseline_filepath)
# rouge = evaluate.load('rouge')
# predictions = baseline_df['t5_extract']
# references = baseline_df['y']
# results = rouge.compute(predictions=predictions,
#                         references=references)
# print(results)

In [ ]:
results_filepath = '/content/drive/MyDrive/266 Final Project/Processed Data/chunk_testing_NL.csv'
results_df = pd.read_csv(results_filepath)

In [ ]:
results_df.head()

,X,y,t5_extract
0,Please ask if you have any questions. Indemnit...,You will indemnify us from all losses and liab...,You will indemnify us from all losses and liab...
1,"We make no warranty as to the reliability, acc...","We make no warranty as to the reliability, acc...","We make no warranty as to the reliability, acc..."
2,This license ends one year after you delete yo...,Regardless of whether you designate content pu...,This license ends one year after you delete yo...
3,You will be notified via email andor a promine...,"For example, we may choose to publish statisti...",You will be notified via email andor a promine...
4,Continuing to use Patreon after a change means...,Continuing to use Patreon after a change means...,"If you are not in the European Union, then the..."


In [ ]:
rouge = evaluate.load('rouge')
predictions = results_df['t5_extract']
references = results_df['y']
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.49027014848709305, 'rouge2': 0.3940601540197298, 'rougeL': 0.4611170335449545, 'rougeLsum': 0.46125682011927027}


In [ ]:
results_df['correct'] = results_df.apply(lambda row: 1 if row['y'] == row['t5_extract'] else 0, axis=1)

In [ ]:
grouped = results_df.groupby(['y']).agg({'X':'count','correct':'sum'}).reset_index()
print('highlights found across more than 1 row:',len(grouped[grouped['X'] > 1]))
print('number of those that are always correct:',len(grouped[(grouped['X'] > 1) & (grouped['correct'] > 1)]))

highlights found across more than 1 row: 89
number of those that are always correct: 16


In [ ]:
results_df[results_df['y'] =='\nWhat do we use your information for?']

,X,y,t5_extract


In [ ]:
results_df[results_df['y'].str.contains('\n')]

,X,y,t5_extract
3,You will be notified via email andor a promine...,"For example, we may choose to publish statisti...",You will be notified via email andor a promine...
13,"ThirdParty Websites, Advertisers or Services \...","Goodreads has no control over,\nand assumes no...","Goodreads has no control over, and assumes no ..."
15,"If You cease using the Service, You confirm th...",DISCLAIMER OF WARRANTIES\nYOU EXPRESSLY UNDERS...,"If You cease using the Service, You confirm th..."
29,Waze\nmay allow you to use the Services whethe...,Waze\nmay allow you to use the Services whethe...,If you choose to use the Waze Services without...
32,We can use information from one device to help...,Social Media Cookies\nOn some pages of our Web...,"Because of how cookies work, we cannot access ..."
...,...,...,...
1117,Learn more about how you can control Advertisi...,Analytics Google Analytics is Googles analytic...,It may use a set of cookies to collect informa...
1118,As permitted under the terms of\nthis Agreemen...,You acknowledge\nthat Goodreads has the perpet...,In the event that Goodreads suspends or termin...
1128,When communicating with our customer support r...,Class action waiver\nWhere permitted under the...,If we feel that your behavior towards any of o...
1158,"By accessing or using the Goodreads\nservices,...",We reserve the right to amend this Agreement a...,We reserve the right to amend this Agreement a...


In [ ]:
results_df.iloc[13]['y']

'Goodreads has no control over,\nand assumes no responsibility for, the content, privacy policies, or practices\nof any third party websites or services.'

In [ ]:
results_df.iloc[13]['y']

'Goodreads has no control over,\nand assumes no responsibility for, the content, privacy policies, or practices\nof any third party websites or services.'

In [ ]:
results_df.iloc[13]['t5_extract']

'Goodreads has no control over, and assumes no responsibility for, the content, privacy policies, or practices of any third party websites or services.'

In [ ]:
# ANYTHING WITH \N IS NOT BEING CLASSIFIED CORRECTLY
print(len(results_df[results_df['y'].str.contains('\n')]))
results_df[results_df['y'].str.contains('\n')]['correct'].sum()

128


0

# Sequence Classification

In [ ]:
# Load the T5 model and tokenizer

# model_name = 't5-base'
model_name = 'google/flan-t5-base'

t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

# t5_model = T5ForConditionalGeneration.from_pretrained(model_name)
t5_model = T5ForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def t5_classification_tokenizer(row, MAX_LENGTH=256):
  input = row['X']
  target = row['y']

  tokenized_input = t5_tokenizer.encode_plus(
      # [input],
      [f'binary classification: {input}'],
      max_length=MAX_LENGTH,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  tokenized_input_ids = tokenized_input['input_ids'][0]
  tokenized_input_attention_mask = tokenized_input['attention_mask'][0]

  # tokenized_target = t5_tokenizer.encode_plus(
  #     [target],
  #     max_length=MAX_LENGTH,
  #     padding='max_length',
  #     truncation=True,
  #     return_attention_mask=True,
  #     return_tensors='pt'
  # )

  # tokenized_target_ids = tokenized_target['input_ids'][0]

  return {'input_ids': tokenized_input_ids,
          'attention_mask': tokenized_input_attention_mask,
          'labels': target}

train_dataset = train_dataset.map(t5_classification_tokenizer)
test_dataset = test_dataset.map(t5_classification_tokenizer)

In [ ]:
batch_size = 5
num_epochs = 3

#save model for fine - tune
dir_path = '/content/drive/MyDrive/266 Final Project/model_checkpoints/'
file_path = dir_path + 't5base-finetuned-seqclassification-chunks'

training_args = TrainingArguments(
    # output_dir='./results',
    output_dir= file_path,          # output directory
    evaluation_strategy="epoch",     # evaluation strategy to adopt during training
    learning_rate=1e-5,              # learning rate
    per_device_train_batch_size=batch_size,   # batch size for training
    per_device_eval_batch_size=batch_size,    # batch size for evaluation
    max_steps = int(num_epochs * num_train_examples / batch_size),
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    report_to="none"        # Important, otherwise will try to connect and write to wandb
)

trainer = Trainer(model=t5_model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.8865, 'grad_norm': 4.87756872177124, 'learning_rate': 9.120647203658109e-06, 'epoch': 0.08793527963418923}
{'loss': 0.6995, 'grad_norm': 9.83422565460205, 'learning_rate': 8.241294407316217e-06, 'epoch': 0.17587055926837847}
{'loss': 0.6976, 'grad_norm': 4.143939971923828, 'learning_rate': 7.361941610974324e-06, 'epoch': 0.2638058389025677}
{'eval_loss': 0.6948984861373901, 'eval_runtime': 92.8875, 'eval_samples_per_second': 25.515, 'eval_steps_per_second': 5.103, 'epoch': 0.3334505803728456}
{'loss': 0.6951, 'grad_norm': 4.644961833953857, 'learning_rate': 6.482588814632431e-06, 'epoch': 1.0182905381639114}
{'loss': 0.6961, 'grad_norm': 1.9751096963882446, 'learning_rate': 5.603236018290539e-06, 'epoch': 1.1062258177981006}
{'loss': 0.6968, 'grad_norm': 1.926625370979309, 'learning_rate': 4.723883221948646e-06, 'epoch': 1.1941610974322898}
{'loss': 0.6958, 'grad_norm': 1.4513710737228394, 'learning_rate': 3.844530425606754e-06, 'epoch': 1.282096377066479}
{'eval_loss': 0.69

TrainOutput(global_step=5686, training_loss=0.7124533316135574, metrics={'train_runtime': 4034.8776, 'train_samples_per_second': 7.046, 'train_steps_per_second': 1.409, 'train_loss': 0.7124533316135574, 'epoch': 2.333098839254309})

In [ ]:
results = trainer.evaluate(eval_dataset = test_dataset) #pass in test data?
print(results)

In [ ]:
import transformers
import torch

# Define task prefix
prefix = "binary classification: "

# Reduce output
transformers.logging.set_verbosity_error()

for test_input in balanced_df[0:3]['doc_text']:
  print(test_input)
  test_input_tokenized = t5_tokenizer([prefix + test_input], return_tensors="pt")

  # decoder_input_ids = t5_tokenizer([''], return_tensors='pt').input_ids.cuda()  # Use empty string as starting point for decoder

  outputs = t5_model(test_input_tokenized['input_ids'].cuda())

  logits = outputs.logits
  probabilities = torch.nn.functional.sigmoid(logits)
  predicted_class = torch.argmax(probabilities, dim=1)
  print('Predicted class:',predicted_class.item())

Apple and its affiliates may share this personal information with each other and use it consistent with this Privacy Policy. They may also combine it with other information to provide and improve our products, services, content, and advertising. You are not required to provide the personal information that we have requested, but, if you chose not to do so, in many cases we will not be able to provide you with our products or services or respond to any queries you may have.
Predicted class: 1
If you did not provide us your personal information directly, we will inform you of the source when we first contact you regarding your candidacy. For research and development purposes, we may use datasets such as those that contain images, voices or other data that could be associated with an identifiable person. When acquiring such datasets, we do so in accordance with applicable law in the jurisdiction in which the dataset is hosted.
Predicted class: 1
Ads that are delivered by Apples advertisin

In [ ]:
def t5_prediction(row):
  # Reduce verbosity
  transformers.logging.set_verbosity_error()

  # Tokenize input text and generate prediction
  row_tokenized = t5_tokenizer(["binary classification: " + row], return_tensors="pt")
  outputs = t5_model(row_tokenized['input_ids'].cuda())
  logits = outputs.logits
  probabilities = torch.nn.functional.sigmoid(logits)
  predicted_class = torch.argmax(probabilities, dim=1)

  return predicted_class.item()

In [ ]:
test_data_df['y_pred'] = test_data_df['X'].apply(t5_prediction)

# Calculate evaluation metrics
y_true = test_data_df['y']
y_pred = test_data_df['y_pred']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.489873417721519
Precision: 0.530530197690587
Recall: 0.5003597891137076
F1 Score: 0.3317000548805817


# Running Predictions on Training Data

In [ ]:
from transformers import T5Tokenizer, TFT5Model, T5ForConditionalGeneration, TrainingArguments, Trainer, T5ForSequenceClassification
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
final_data_folder = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/'
training_data = pd.read_csv(final_data_folder + 'chunk_training_data.csv')
testing_data = pd.read_csv(final_data_folder + 'chunk_testing_data.csv')

In [ ]:
# Load in tokenizer and checkpointed model
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

dir_path = '/content/drive/MyDrive/266 Final Project/model_checkpoints'
file_path = dir_path + 't5base-finetuned-seqclassification'
t5_model = T5ForConditionalGeneration.from_pretrained(file_path + '/checkpoint-3789') #add path name of checkpoint you want to load
t5_model = t5_model.to('cuda')

In [ ]:
def classify_t5_summary(text):
    transformers.logging.set_verbosity_error()
    # Tokenize the input text
    test_inputs = t5_tokenizer([f'binary classification: {text}'], return_tensors='pt')
    test_output_ids = t5_model.generate(test_inputs['input_ids'].cuda())
    return [t5_tokenizer.decode(out_ids, skip_special_tokens = True, clean_up_tokenization_spaces = False) for out_ids in test_output_ids]

In [ ]:
training_data['pred'] = training_data['doc_text'].apply(lambda x: classify_t5_summary(x))
training_data['pred_label'] = training_data['pred'].apply(lambda x: 1 if x[0] == 'Important' else 0)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# Calculate evaluation metrics on training data
y_true = training_data['label']
y_pred = training_data['pred_label']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5445234113712375
Precision: 0.5856003749981098
Recall: 0.5445234113712375
F1 Score: 0.4824322338997251
